In [2]:
# To handle relative importing
import os
import sys
module_paths = [
    os.path.abspath(os.path.join('..')),
]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

In [3]:
%load_ext autoreload
%autoreload 2
# import necessary files
from tqdm import tqdm
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
# dataloader
from src.dataloaders.dataloader import ImgNetDataset
from torch.utils.data import DataLoader

In [6]:
# json file of our validation set
# relabel_json = json.load(open('/root/CS570-Final-Project/datasets/imgnet1k_original/val_relabelled.json', 'r')) # our relabeled validation set
relabel_json = json.load(open('/root/CS570-Final-Project/datasets/imgnet1k_original/val.json', 'r')) # original validation set
# json file of ReaL validation set
real_json = json.load(open('/root/CS570-Final-Project/datasets/imgnet1k_ReaL/val.json', 'r'))

In [8]:
# check how much the labels per sample align
labels_dict_relabel = {}
for sample in relabel_json['data']:
    labels_dict_relabel[sample['img_path']] = sample['labels']
labels_dict_real = {}
for sample in real_json['data']:
    labels_dict_real[sample['img_path']] = sample['labels']

## Some Notes:
- we skip the samples in ReaL that has not beed assigned any labels

## Compute Average Jaccard Similarity

In [11]:
avg_jaccard_sim = 0
cnt = 0
for key in labels_dict_relabel.keys():
    labels_relabel = set(labels_dict_relabel[key])
    labels_real = set(labels_dict_real[key])
    if len(labels_real) == 0:
        continue
    jaccard_sim = len(labels_relabel.intersection(labels_real)) / len(labels_relabel.union(labels_real))
    avg_jaccard_sim += jaccard_sim
    cnt += 1
avg_jaccard_sim /= cnt
print(avg_jaccard_sim)

0.8234973638735616


## Compute Average Precision / Recall

In [12]:
avg_prec, avg_recall = 0, 0
cnt = 0
for key in labels_dict_relabel.keys():
    labels_relabel = set(labels_dict_relabel[key])
    labels_real = set(labels_dict_real[key])
    if len(labels_real) == 0:
        continue
    precision = len(labels_relabel.intersection(labels_real)) / len(labels_relabel)
    recall = len(labels_relabel.intersection(labels_real)) / len(labels_real)
    avg_prec += precision
    avg_recall += recall
    cnt += 1
avg_prec /= cnt
avg_recall /= cnt
avg_prec, avg_recall

(0.9002284518649786, 0.8234973638735616)

## Compute Average F1 score

In [14]:
avg_f1 = 0
cnt = 0
for key in labels_dict_relabel.keys():
    labels_relabel = set(labels_dict_relabel[key])
    labels_real = set(labels_dict_real[key])
    if len(labels_real) == 0:
        continue
    precision = len(labels_relabel.intersection(labels_real)) / len(labels_relabel)
    recall = len(labels_relabel.intersection(labels_real)) / len(labels_real)
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        f1 = 0
    avg_f1 += f1
    cnt += 1
avg_f1 /= cnt
avg_f1

0.846062941691404

## Misc

In [4]:
# json file of our validation set
relabel_json = json.load(open('/root/CS570-Final-Project/datasets/imgnet1k_original/val.json', 'r')) # original validation set
# json file of ReaL validation set
real_json = json.load(open('/root/CS570-Final-Project/datasets/imgnet1k_ReaL/val.json', 'r'))

In [5]:
labels_dict_relabel = {}
for sample in relabel_json['data']:
    labels_dict_relabel[sample['img_path']] = sample['labels']
labels_dict_real = {}
for sample in real_json['data']:
    labels_dict_real[sample['img_path']] = sample['labels']

# check whether validation set is a subset of ReaL validation set
cnt = 0
for key in labels_dict_relabel.keys():
    labels_relabel = set(labels_dict_relabel[key])
    labels_real = set(labels_dict_real[key])
    if labels_relabel.issubset(labels_real):
        cnt += 1
cnt

42164